In [1]:
import numpy as np

#import matplotlib.pyplot as plt
#from bokeh.io import push_notebook, show, output_notebook; 
#from bokeh.plotting import figure; 
#from bokeh.layouts import column
#output_notebook()

import USGShydro


to do:

USGShydro.py
- ~~ownload rainfall~~ 
- ~~download runoff~~
- merge data capability?
- convert data to numpy array

RainfallRunoff.py
- create time series dataset of lags, etc.
- drop observations with missing values
- run rainfall-runoff model
- plot predicted runoff vs. actual
- predict days above flood stage with models trained over different time periods

-universalize the procedure for other sites
-- create tool to query sites for dates and availability of needed series
-compare sites and see if model relates to hydrologic featues, basin topography, landcover, etc.

In [2]:
site='sites=02314495'
start='startDT=1951-01-01T00:00%2b0000'
start='startDT=2019-05-18T00:00%2b0000'#short interval for practice
end='endDT=2019-05-19T06:56-0400'
#data1=getUSGS.Get_data(site, start, end)
paramlist=['00065', '00045'] #must be entered as strings
modelfeatures=[]
try1=USGShydro.Hydrositedatamodel(site,start,end,paramlist,modelfeatures)

a
b


In [3]:
try1.get_data()

In [95]:
namespace={'ns0':"http://www.opengis.net/waterml/2.0",
           'ns1':"http://www.opengis.net/gml/3.2",
           'ns3':"http://www.w3.org/1999/xlink",
           'ns4':"http://www.opengis.net/om/2.0"}
extracted=[];j=-1
for elem in try1.root.findall('ns0:observationMember',namespace):
    for elem2 in elem.findall('ns4:OM_Observation',namespace):
        #print(elem2.tag)
        j+=1;i=-1
        extracted.append([])
        for elem3 in elem2.findall('ns4:result',namespace):
            for elem4 in elem3.findall('ns0:MeasurementTimeseries',namespace):
                for elem5 in elem4.findall('ns0:point',namespace):
                    for elem6 in elem5.findall('ns0:MeasurementTVP',namespace):
                        time=elem6.find('ns0:time',namespace)
                        val=elem6.find('ns0:value',namespace)
                        for _ in range(len(times)):
                            i+=1
                            print('j=',j,'i=',i)
                            print(len(time),len(val))
                            extracted[j].append((time.text,val.text))

    
    

j= 0 i= 0
0 0
j= 0 i= 1
0 0
j= 0 i= 2
0 0
j= 0 i= 3
0 0
j= 0 i= 4
0 0
j= 0 i= 5
0 0
j= 0 i= 6
0 0
j= 0 i= 7
0 0
j= 0 i= 8
0 0
j= 0 i= 9
0 0
j= 0 i= 10
0 0
j= 0 i= 11
0 0
j= 0 i= 12
0 0
j= 0 i= 13
0 0
j= 0 i= 14
0 0
j= 0 i= 15
0 0
j= 0 i= 16
0 0
j= 0 i= 17
0 0
j= 0 i= 18
0 0
j= 0 i= 19
0 0
j= 0 i= 20
0 0
j= 0 i= 21
0 0
j= 0 i= 22
0 0
j= 0 i= 23
0 0
j= 0 i= 24
0 0
j= 0 i= 25
0 0
j= 0 i= 26
0 0
j= 0 i= 27
0 0
j= 0 i= 28
0 0
j= 0 i= 29
0 0
j= 0 i= 30
0 0
j= 0 i= 31
0 0
j= 0 i= 32
0 0
j= 0 i= 33
0 0
j= 0 i= 34
0 0
j= 0 i= 35
0 0
j= 0 i= 36
0 0
j= 0 i= 37
0 0
j= 0 i= 38
0 0
j= 0 i= 39
0 0
j= 0 i= 40
0 0
j= 0 i= 41
0 0
j= 0 i= 42
0 0
j= 0 i= 43
0 0
j= 0 i= 44
0 0
j= 0 i= 45
0 0
j= 0 i= 46
0 0
j= 0 i= 47
0 0
j= 0 i= 48
0 0
j= 0 i= 49
0 0
j= 0 i= 50
0 0
j= 0 i= 51
0 0
j= 0 i= 52
0 0
j= 0 i= 53
0 0
j= 0 i= 54
0 0
j= 0 i= 55
0 0
j= 0 i= 56
0 0
j= 0 i= 57
0 0
j= 0 i= 58
0 0
j= 0 i= 59
0 0
j= 0 i= 60
0 0
j= 0 i= 61
0 0
j= 0 i= 62
0 0
j= 0 i= 63
0 0
j= 0 i= 64
0 0
j= 0 i= 65
0 0
j= 0 i= 66
0 0
j= 0 

In [98]:
matched=[]
m=[[] for _ in range(i)] #this will track successful matches across value series, it will be i+1 by j+1

for jj in range(j-1): #j+1 is the number of different value series. get it working for 2 now. need to research algorithms
    for ii in range(i): 
        if extracted[0][ii][0]==extracted[jj+1][ii][0]:
            m[ii].append('yes',jj)
            else m[ii].append('no',jj)

            #matched[m].append((extracted[0][ii][0])) #start the tuple for each date
            #matched[m][]
        

SyntaxError: invalid syntax (<ipython-input-98-db647170f567>, line 8)